In [12]:
import array, gzip, StringIO, urllib, urllib2

def benchmark(label, data):
    zipped = StringIO.StringIO()
    zipper = gzip.GzipFile(fileobj = zipped, mode='wb')
    zipper.write(data)
    zipper.close()
    print "%.2fM / %.2fM: %s" % (len(zipped.getvalue()) / 1e6, len(data) / 1e6, label)

In [13]:
xyz_time_bin = open('viirs-2014.timev').read()
benchmark("xyz_time float row-major", xyz_time_bin)
xyz_time = array.array('f', xyz_time_bin)

18.41M / 26.06M: Original, xyz_time float row-major


In [14]:
lonlat_time_temp_bin = urllib2.urlopen('https://data.cmucreatelab.org/createlab-viirs/createlab-viirs-uncorrected-20140314-20150403.bin').read()
benchmark("lonlat_time_temp float row-major", lonlat_time_temp_bin)
lonlat_time_temp = array.array('f', lonlat_time_temp_bin)

15.16M / 26.06M: Original lonlat float row-major


In [22]:
lonlat_time = []
for i in range(0, len(lonlat_time_temp), 4):
    lonlat_time.append(lonlat_time_temp[i + 0])
    lonlat_time.append(lonlat_time_temp[i + 1])
    lonlat_time.append(lonlat_time_temp[i + 2])
lonlat_time_bin = array.array('f', lonlat_time).tostring()
benchmark("lonlat_time float row-major", lonlat_time_bin)

11.30M / 19.54M: lonlat_time float row-major


In [19]:
lonlat_time[0:3]

[171.02980041503906, 61.905250549316406, 1394773721088.0]

19542672